In [ ]:
!pip install transformers==4.1.1 sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numpy
import numpy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
target_model_name = 'Helsinki-NLP/opus-mt-fr-en'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-fr-en and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
en_model_name = 'Helsinki-NLP/opus-mt-en-fr'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-fr and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts, return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [ ]:
def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, en_model, en_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts,  target_model, target_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

In [ ]:
import re
file_t = open('medline_en2fr_fr.txt')
lines = file_t.readlines()
lines = [re.sub('doc.*\\t.*\\t', '', line) for line in lines]
lines

['Contexte\n',
 'L’hypertension intracrânienne (HTIC) idiopathique est une affection rare qui touche surtout les femmes atteintes d’obésité en âge de procréer.\n',
 'Les symptômes courants sont des maux de tête, des troubles de la vue, des acouphènes pulsatiles et un oedème papillaire.\n',
 'Le lien entre l’HTIC idiopathique et l’obésité est bien établi dans la littérature, ce qui suggère que la perte de poids pourrait améliorer le tableau clinique de l’HTIC.\n',
 'Pour les patients atteints d’obésité sévère pour lesquels le traitement conservateur ne fonctionne pas, la chirurgie bariatrique est un moyen efficace de perdre du poids.\n',
 'Cette revue systématique de la littérature vise à déterminer l’efficacité de la chirurgie bariatrique dans le traitement de l’HTIC idiopathique.\n',
 'Méthodes\n',
 'Nous avons interrogé MEDLINE, Embase, Scopus, la Bibliothèque Cochrane et Web of Science (limites : études portant sur les humains publiées en anglais entre janvier 1946 et juillet 2015).

In [ ]:
final_text = []
for start in range(0, len(lines), 50):
  cut = lines[start:start+50]
  tr_cut = translate(cut,  target_model, target_tokenizer, 
                         language='fr')
  final_text.append(tr_cut)
  

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [ ]:
final_text

[['Risk factors associated with deterioration of the results for VOCID-19: retrospective study in Saudi Arabia.',
  'The rapid emergence of the 2019 Coronavirus disease (COVID-19) has led to the infection of millions of patients and hundreds of thousands of deaths worldwide.',
  'The delivery of health care services is being undermined by the massive increase in the number of patients infected during this pandemic.',
  'The purpose of this study was to assess risk factors associated with poor prognosis in patients with VOCID-19 in Saudi Arabia.',
  'This was a multicentre retrospective cohort study that included all laboratory-confirmed cases of VOCID-19, with final results in Saudi Arabia in March 2020.',
  'The demographics, clinical history, co-morbidity and results were extracted from the National Health Electronic Surveillance Network database.',
  'We used logistic regression models to calculate crude and adjusted Odd ratios (OR) to study risk factors for critical outcomes (i.e. 

In [ ]:
import json
json_file = json.dumps(final_text)
json_file

'[["Risk factors associated with deterioration of the results for VOCID-19: retrospective study in Saudi Arabia.", "The rapid emergence of the 2019 Coronavirus disease (COVID-19) has led to the infection of millions of patients and hundreds of thousands of deaths worldwide.", "The delivery of health care services is being undermined by the massive increase in the number of patients infected during this pandemic.", "The purpose of this study was to assess risk factors associated with poor prognosis in patients with VOCID-19 in Saudi Arabia.", "This was a multicentre retrospective cohort study that included all laboratory-confirmed cases of VOCID-19, with final results in Saudi Arabia in March 2020.", "The demographics, clinical history, co-morbidity and results were extracted from the National Health Electronic Surveillance Network database.", "We used logistic regression models to calculate crude and adjusted Odd ratios (OR) to study risk factors for critical outcomes (i.e. admission t

In [ ]:
with open('translate.json', 'w') as file:
  file.write(json_file) 